<a href="https://colab.research.google.com/github/iGhostlp/Albus/blob/Sin-Dientes/Proyecto_BBVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Armado de entorno

In [36]:
pip install docker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.3.2-bin-hadoop3'

In [40]:
import findspark

In [41]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [42]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

In [43]:
ls

Dataset/   sample_data/              spark-3.3.2-bin-hadoop3.tgz
Datasets/  spark-3.3.2-bin-hadoop3/  spark-3.3.2-bin-hadoop3.tgz.1


In [44]:
!tar xf spark-3.3.2-bin-hadoop3.tgz

In [45]:
!java -version

openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


In [46]:
!readlink -f $(which java) | sed "s:bin/java::"

/usr/lib/jvm/java-11-openjdk-amd64/


In [47]:
!echo $JAVA_HOME

/usr/lib/jvm/java-11-openjdk-amd64


In [48]:
!pip install -q findspark

In [49]:
!pip install pyspark 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [50]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.3.2-bin-hadoop3'

In [51]:
import findspark

In [52]:
findspark.init()

In [53]:
findspark.find()

'/content/spark-3.3.2-bin-hadoop3'

In [54]:
from pyspark.sql import SparkSession

In [55]:
from pyspark.sql.functions import col


In [56]:
spark = SparkSession.builder.master("local[*]").config('spark.sql.parquet.datetimeRebaseModeInRead','CORRECTED').getOrCreate()

In [57]:
spark.sparkContext.setLogLevel("ERROR")

In [58]:
spark

In [59]:
import pandas as pd

# Creación de dataframes

In [60]:
# Creando el dataframe

In [61]:
df_address = spark.read.parquet("/content/Dataset/address.snappy.parquet")

In [62]:
df_customer = spark.read.parquet("/content/Dataset/customer.snappy.parquet")

In [63]:
df_customer_b = spark.read.parquet("/content/Dataset/customer_basics.snappy.parquet")

In [64]:
df_mails = spark.read.parquet("/content/Dataset/email.snappy.parquet")

In [65]:
df_phones = spark.read.parquet("/content/Dataset/phones.snappy.parquet")

# Exploración de la tabla

In [66]:
# Exploracion de tabla

df_mails.show()

+-----------+---------+-------------------+--------------+----------+------------------+--------------------+-----------------+--------------+-------------------+----------------+--------------------------+--------------+----------------+-------------------+--------------------+-----------------------+---------------------+--------------------------+-------------------+----------------+
|customer_id|role_type|address_sequence_id|residence_type|email_type|primary_email_type|          email_desc|email_domain_type|encripted_type|field_length_number|   comments_desc|customer_email_status_type|email_app_type|register_user_id|last_change_user_id|last_change_hms_date|last_change_terminal_id|operational_load_date|customer_email_status_date|registry_entry_date|last_change_date|
+-----------+---------+-------------------+--------------+----------+------------------+--------------------+-----------------+--------------+-------------------+----------------+--------------------------+--------------

# QUERY

In [67]:
# query solicitada

df_customer_mails = df_customer_b.join(df_mails, ['customer_id'], how='left')

In [68]:
#exploracion esquema

df_customer_mails.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- personal_type: string (nullable = true)
 |-- personal_id: string (nullable = true)
 |-- admission_date: date (nullable = true)
 |-- sale_channel_id: string (nullable = true)
 |-- operation_channel_id: string (nullable = true)
 |-- branch_id: integer (nullable = true)
 |-- business_area_id: integer (nullable = true)
 |-- campaing_id: decimal(2,0) (nullable = true)
 |-- customer_situation_type: integer (nullable = true)
 |-- customer_condition_type: integer (nullable = true)
 |-- manager_id: integer (nullable = true)
 |-- tracking_manager_id: string (nullable = true)
 |-- credit_manager_id: string (nullable = true)
 |-- credit_recovery_manager_id: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- second_last_name: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- marital_status_type: integer (nullable = true)
 |-- gender_type: string (nullable = true)
 |-- birth_date: date (nullable = true)
 |

In [69]:
df_customer_mails.show()

+-----------+-------------+-------------+--------------+---------------+--------------------+---------+----------------+-----------+-----------------------+-----------------------+----------+-------------------+-----------------+--------------------------+----------+----------------+-----------------+-------------------+-----------+----------+----------------------+--------------------+---------------------+-----------+--------------------------+----------------------------+-------------------------------+------------------------+------------------------+--------------------------+------------------------+--------------------------+----------------------------+------------------+------------------------+-------------------+-------------------+---------------------------+-----------------+-------------------+-----------------+-------------------------------+--------------------+---------------------+------------------+----------------+--------------------+--------------------------+----

# Extraccion de datos

In [70]:
df_emails = spark.read.parquet('/content/Datasets/email.snappy.parquet')

In [76]:
df_customer = spark.read.parquet('/content/Datasets/customer_basics.snappy.parquet')

In [77]:

df_customer_emails = df_emails.join(df_customer, 'customer_id', how="right")

#Filtrado de e-mails

In [83]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat,col, row_number, desc, collect_list
from pyspark.sql.window import Window

In [78]:
df_emails_cut = df_emails.drop('role_type', 'email_type','address_sequence_id','residence_type','primary_email_type','email_domain_type','encripted_type','field_length_number','comments_desc','customer_email_status_type','email_app_type','register_user_id','last_change_user_id','last_change_hms_date','last_change_terminal_id','operational_load_date','customer_email_status_date','registry_entry_date')


In [84]:
df_emails_sorted = df_emails_cut.orderBy([df_emails_cut.customer_id, desc('last_change_date')])


In [85]:
df_emails_sorted.toPandas()


,customer_id,email_desc,last_change_date
0,00000000,ALVAROJOSE.DAGAND@BBVA.COM,2019-12-18
1,00000000,LIBRERIAURUGUAY@GMAIL.COM,2015-10-27
2,00027555,ALEFRID666@YAHOO.COM.AR,2012-07-08
3,00027568,SCATIVA@GMAIL.COM,2016-03-15
4,00027595,NO@GMAIL.COM,2018-08-07
...,...,...,...
495,27514366,NICOLAS.SZEBUN@HOTMAIL.COM,2021-01-25
496,27514374,TOMASBORBAS@HOTMAIL.COM,2021-01-25
497,27514379,FSANFRANCISCO@GMAIL.COM,2021-01-26
498,27514380,JUANCA.BIANCU@GMAIL.COM,2021-01-25


In [86]:
window = Window.partitionBy(df_emails_sorted.customer_id).orderBy(desc(df_emails_sorted.last_change_date))


In [87]:
df_email = df_emails_sorted.withColumn('row_num', row_number().over(window))


In [88]:
df_email = df_email.filter(df_email.row_num <= 3)


In [89]:
df_email.toPandas()

,customer_id,email_desc,last_change_date,row_num
0,00000000,ALVAROJOSE.DAGAND@BBVA.COM,2019-12-18,1
1,00000000,LIBRERIAURUGUAY@GMAIL.COM,2015-10-27,2
2,00027555,ALEFRID666@YAHOO.COM.AR,2012-07-08,1
3,00027568,SCATIVA@GMAIL.COM,2016-03-15,1
4,00027595,NO@GMAIL.COM,2018-08-07,1
...,...,...,...,...
495,27514366,NICOLAS.SZEBUN@HOTMAIL.COM,2021-01-25,1
496,27514374,TOMASBORBAS@HOTMAIL.COM,2021-01-25,1
497,27514379,FSANFRANCISCO@GMAIL.COM,2021-01-26,1
498,27514380,JUANCA.BIANCU@GMAIL.COM,2021-01-25,1


In [90]:
df_pivot_email = df_email.groupBy('customer_id').agg(collect_list('email_desc').alias('last_3_changes_list'))

In [91]:
df_pivot_email = df_pivot_email.selectExpr('customer_id', 'last_3_changes_list[0] as Email_1', 'last_3_changes_list[1] as Email_2', 'last_3_changes_list[2] as Email_3')

In [92]:
df_pivot_email.toPandas()

,customer_id,Email_1,Email_2,Email_3
0,00000000,ALVAROJOSE.DAGAND@BBVA.COM,LIBRERIAURUGUAY@GMAIL.COM,None
1,00027555,ALEFRID666@YAHOO.COM.AR,None,None
2,00027568,SCATIVA@GMAIL.COM,None,None
3,00027595,NO@GMAIL.COM,None,None
4,00027609,lindocampo@yahoo.com,None,None
...,...,...,...,...
424,27514366,NICOLAS.SZEBUN@HOTMAIL.COM,None,None
425,27514374,TOMASBORBAS@HOTMAIL.COM,None,None
426,27514379,FSANFRANCISCO@GMAIL.COM,None,None
427,27514380,JUANCA.BIANCU@GMAIL.COM,None,None


In [93]:
df_pivot_email = df_pivot_email.na.fill('---')

In [94]:
df_pivot_email.toPandas()

,customer_id,Email_1,Email_2,Email_3
0,00000000,ALVAROJOSE.DAGAND@BBVA.COM,LIBRERIAURUGUAY@GMAIL.COM,---
1,00027555,ALEFRID666@YAHOO.COM.AR,---,---
2,00027568,SCATIVA@GMAIL.COM,---,---
3,00027595,NO@GMAIL.COM,---,---
4,00027609,lindocampo@yahoo.com,---,---
...,...,...,...,...
424,27514366,NICOLAS.SZEBUN@HOTMAIL.COM,---,---
425,27514374,TOMASBORBAS@HOTMAIL.COM,---,---
426,27514379,FSANFRANCISCO@GMAIL.COM,---,---
427,27514380,JUANCA.BIANCU@GMAIL.COM,---,---
